In [ ]:
import os
import json
import praw
import time
from tqdm import tqdm
from datetime import datetime

# API credentials 

reddit = praw.Reddit(
    client_id='Y5DdpLJDcxJPSAT7Vvmo-A',
    client_secret='KZ83gqe2B-C3uJwlcjEI4e1gR_gpPw',
    user_agent='test by u/Inner-Astronomer3735'
)

# constants 
max_depth = 30
base_folder = 'thread_tests'


if not os.path.exists(base_folder):
    os.makedirs(base_folder)

def fetch_comments(comment_forest, depth=0):

    if depth > max_depth:
        return []
    
    comment_trees = []

    # loop through root nodes
    for comment in comment_forest:
        
        # When there's too many comments the API collaposes some of them into MoreComments objects
        # This bit of code is to expand those and get the comments. 
        if isinstance(comment, praw.models.MoreComments):
            time.sleep(0.3)
            # this is an API call (time.sleep with rate limit in mind, which is 100 API calls per minute I think)
            more = comment.comments()
            comment_trees.extend(fetch_comments(more, depth))
            continue

        dictionary = {
                    "author": str(comment.author),
                    "body": comment.body,
                    "score": comment.score,
                    "replies": fetch_comments(comment.replies, depth+1)
                }
        
        comment_trees.append(dictionary)

    return comment_trees

def fetch_post(post_id): 

    post = reddit.submission(id = post_id)
    comments_thread = post.comments

    comments_processed = fetch_comments(comments_thread)

    # create file path
    timestamp = datetime.now().strftime("%Y-%m-%d %H-%M")

    subreddit_folder = os.path.join(base_folder, post.subreddit.display_name)
    timestamp_folder = os.path.join(subreddit_folder, f'timestamp -- {timestamp}')

    if not os.path.exists(timestamp_folder):
        os.makedirs(timestamp_folder)

    post_file = os.path.join(timestamp_folder, f'id -- {post.id}.json, timestamp -- {timestamp}.json')


    # gather post data
    post_data = {
                "title": post.title,
                "author": str(post.author),
                "subreddit": post.subreddit.display_name,
                "score": post.score,
                "upvote_ratio": post.upvote_ratio,
                "num_comments (reported by reddit)": post.num_comments,
                "url": post.url,
                "id": post.id,
                "selftext": post.selftext,
                "comments": comments_processed
            }
    
    with open(post_file, "w", encoding="utf-8") as file:
        json.dump(post_data, file, indent=4)
        print(f'Saved {post.id}.json')

# Apparently n_posts has to be less than 100. 
def fetch_subreddit(subreddit_name, n_posts): 

    subreddit = reddit.subreddit(subreddit_name)
    post_ids = [post.id for post in subreddit.top(limit=n_posts)]

    for post_id in tqdm(post_ids): 
        fetch_post(post_id)

In [ ]:
# test
fetch_subreddit('python', 3)

 33%|███▎      | 1/3 [00:02<00:05,  2.68s/it]

Saved g53lxf.json


 67%|██████▋   | 2/3 [00:08<00:04,  4.63s/it]

Saved hoolsm.json


100%|██████████| 3/3 [00:10<00:00,  3.47s/it]

Saved gftejm.json
